<a href="https://colab.research.google.com/github/yahyasungur/nlp_dl_ml_projects/blob/master/IMDB%2C_and_Amazon_Review_Classification_with_SpaCy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importting the datasets

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_yelp = pd.read_csv('yelp_labelled.txt', sep='\t', header=None)

In [3]:
data_yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
#Assign column name
column_name = ['Review','Sentiment']
data_yelp.columns = column_name

In [5]:
data_yelp.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [7]:
data_yelp.shape

(1000, 2)

In [8]:
data_amazon = pd.read_csv('amazon_cells_labelled.txt',sep='\t',header=None)
data_amazon.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [9]:
data_amazon.columns = column_name

In [10]:
data_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [11]:
data_amazon.shape

(1000, 2)

In [12]:
data_imdb = pd.read_csv('imdb_labelled.txt',sep='\t',header=None)
data_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [13]:
data_imdb.columns = column_name

In [14]:
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [15]:
data_imdb.shape

(748, 2)

In [16]:
# Append all the data in a single dataframe
data = data_yelp.append([data_amazon,data_imdb],ignore_index=True)

In [17]:
data.shape

(2748, 2)

In [18]:
data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [19]:
# Distributions of sentiments
data['Sentiment'].value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [20]:
# check for null values
data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

In [21]:
x = data['Review']
y = data['Sentiment']

#Data Cleaning

In [22]:
# removing stopwords, puctiations & apply lemmatization
import string

In [23]:
punct = string.punctuation

In [24]:
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [25]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS) #list of stopwords

In [29]:
# create a function for data cleaning
import spacy
nlp = spacy.load('en_core_web_sm')

def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens = []
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)

  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  
  return cleaned_tokens

In [30]:
text_data_cleaning(" Hello all, It's a beautiful day outside there!!")

['hello', 'beautiful', 'day', 'outside']

##Vectorization Feature Eng. (TF-IDF)

In [31]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [32]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)
# tokenization will be done according to this function

In [34]:
classifier = LinearSVC()

#Train the model

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [36]:
x_train.shape, x_test.shape

((2198,), (550,))

In [37]:
x_train.head()

2572    An Italian reviewer called this "a small, grea...
526                          And it was way to expensive.
1509    As an earlier review noted, plug in this charg...
144     Nice blanket of moz over top but i feel like t...
2483    The film gives meaning to the phrase, "Never i...
Name: Review, dtype: object

## Fit the x_train and y_train

In [38]:
# it'll first do vectorization and then classification
clf = Pipeline([('tfidf',tfidf) , ('clf',classifier)])

In [39]:
clf.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_data_cleaning at 0x7f2ddcc999e0>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept

#Predict the Test Set Results

In [40]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [41]:
y_pred = clf.predict(x_test)

In [42]:
#confusion_matrix
confusion_matrix(y_test, y_pred)

array([[201,  78],
       [ 50, 221]])

In [43]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.72      0.76       279
           1       0.74      0.82      0.78       271

    accuracy                           0.77       550
   macro avg       0.77      0.77      0.77       550
weighted avg       0.77      0.77      0.77       550



In [44]:
accuracy_score(y_test, y_pred)

0.7672727272727272

In [45]:
# if output will be 1, that means review is positive

clf.predict(["Wow, I am learning NLP in fun fashion!"])

array([1])

In [55]:
# if output will be 0, that means review is negative
clf.predict(["Hard to learn new things!"])

array([0])